# Afriat

In [1]:
import warnings
import os
import sys
import gc

In [ ]:
import warnings
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import tqdm
import scipy.stats
import sklearn
from sklearn.dummy import DummyClassifier

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis
from tardis._utils.functions import label_transfer
from tardis._utils.warnings import suppress_stdout, ignore_predetermined_warnings
tardis.config = tardis.config_server
print(f"CUDA used: {torch.cuda.is_available()}")

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/home/icb/kemal.inecik/work/codes/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [ ]:
num_ood = 1  # not functioning

In [ ]:
def get_availables():
    adata_file_path = os.path.join(tardis.config.io_directories["processed"], "biolord_afriat.h5ad")
    assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
    adata = ad.read_h5ad(adata_file_path)
    df = adata.obs.groupby(['time_cat', 'zone', 'status_control']).size()
    availables = list(df[df>100].index)
    return availables

In [ ]:
def get_classification(ood_seed, target_obs):
    with warnings.catch_warnings(), suppress_stdout():
        warnings.simplefilter("ignore")
        adata_file_path = os.path.join(tardis.config.io_directories["processed"], "biolord_afriat.h5ad")
        adata = ad.read_h5ad(adata_file_path)
        df = adata.obs.groupby(['time_cat', 'zone', 'status_control']).size()
        availables = list(df[df>100].index)
        def adata_ood_creator(adata, seed, size):
            rng = np.random.default_rng(seed=seed)
            # random_choices = rng.choice(np.arange(len(availables)), size=size, replace=False)
            # choices = [availables[i] for i in random_choices]
            choices = [availables[ood_seed]]
            choices_bool = np.vstack([(adata.obs["time_cat"] == choice[0]) & (adata.obs["zone"] == choice[1]) & (adata.obs["status_control"] == choice[2]) for choice in choices]).any(axis=0)
            return adata[~choices_bool, :].copy(), choices_bool, choices[0]
        
        adata, choices_bool, choice = adata_ood_creator(adata, seed=ood_seed, size=num_ood)
    
        dir_path = os.path.join(
            tardis.config.io_directories["models"],
            f"afriat_v2_4_1_ood-seed-{ood_seed}_size-{num_ood}"
        )
        vae = tardis.MyModel.load(dir_path, adata=adata)
        from tardis._disentanglementmanager import DisentanglementManager as DM
    
        adata_main = ad.read_h5ad(adata_file_path)
        adata_main.obs["ood"] = choices_bool.astype(int).astype(str)
        adata_main.obs["ood"] = adata_main.obs["ood"].astype("category")
        adata_ood = adata_main[adata_main.obs["ood"] == "1"].copy()
    
        k = target_obs
        
        latent = ad.AnnData(X=vae.get_latent_representation(adata)[:, DM.configurations.get_by_obs_key(k).reserved_latent_indices], obs=adata.obs.copy())
        sc.pp.neighbors(latent)
        
        latent_ood = ad.AnnData(X=vae.get_latent_representation(adata_ood)[:, DM.configurations.get_by_obs_key(k).reserved_latent_indices], obs=adata_ood.obs.copy())
        sc.pp.neighbors(latent_ood)
        sc.tl.umap(latent_ood)
        
        label_transfer(latent, latent_ood, cluster_list=[k], neighbors_count=30)
        predicted = latent_ood.obs[f"transf_{k}"].astype(str).values
        true = latent_ood.obs[k].astype(str).values

        t = latent.obs[k].astype(str).to_numpy().copy()
        dummy_clf = DummyClassifier(strategy="stratified")
        dummy_clf.fit(t.reshape(-1, 1), t)
        dummy_predictions = dummy_clf.predict(t.reshape(-1, 1))
        
        result = {
            "accuracy": sklearn.metrics.accuracy_score(true, predicted), 
            "baseline": sklearn.metrics.accuracy_score(t, dummy_predictions)
        }
        result.update({f"OOD_{i}": choice[ind] for ind, i in enumerate(['time_cat', 'zone', 'status_control'])})
        return result
        

In [ ]:
result = []
for z in ['time_cat', 'zone', 'status_control']:
    for i in tqdm.tqdm(len(get_availables()), desc=z, ncol=100):
        d = get_classification(i, z)
        d.update({"target_covariate": z})
        result.append(d)

In [ ]:
df = pd.DataFrame(result)
df